In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import scipy.io
from pathlib import Path

# Load the .mat file
data_path = '/home/chanae/thesis/nn_predicted_fc/predicted_fc.mat'
data = scipy.io.loadmat(data_path) 

# Extract the matrices
sc_matrices = data['in']
efc_matrices = data['out']
pfc_matrices = data['predicted']

# Initialize containers for the corrected matrices
efc_matrices_residuals = np.empty(efc_matrices.shape)
pfc_matrices_residuals = np.empty(pfc_matrices.shape)

# Iterate over all matrices
for i in range(efc_matrices.shape[0]):  # iterate through each of the matrices

    # Flatten the connectivity data and prepare for regression
    X = sc_matrices[i].flatten()
    Y_efc = efc_matrices[i].flatten()
    Y_pfc = pfc_matrices[i].flatten()

    # Add a constant (intercept term) to the predictors
    X = sm.add_constant(X)

    # Perform the regression for empirical functional connectivity
    model_efc = sm.OLS(Y_efc, X)
    results_efc = model_efc.fit()
    residuals_efc = results_efc.resid

    # Perform the regression for predicted functional connectivity
    model_pfc = sm.OLS(Y_pfc, X)
    results_pfc = model_pfc.fit()
    residuals_pfc = results_pfc.resid

    # Reshape the residuals back into the original matrix shape
    residuals_efc_matrix = residuals_efc.reshape(efc_matrices[i].shape)
    residuals_pfc_matrix = residuals_pfc.reshape(pfc_matrices[i].shape)

    # Store the corrected matrices
    efc_matrices_residuals[i] = residuals_efc_matrix
    pfc_matrices_residuals[i] = residuals_pfc_matrix

# Save the corrected matrices into a new .mat file
save_path = Path('/home/chanae/thesis/residual_matrices')
scipy.io.savemat(save_path/'residual_matrices.mat', {'in': sc_matrices, 'out': efc_matrices_residuals, 'predicted': pfc_matrices_residuals})